In [1]:
# all codes come are changed based on LDR code.
import argparse
import torch.optim as optim
import os
import yaml
from mcrgan.default import _C as config
from mcrgan.default import update_config
import torch
from mcrgan.trainer import MCRTrainer
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.models import get_BNN_models
from mcrgan.models import get_noise
from mcrgan.models import DiscriminatorMNIST
from mcrgan.models import weights_init_mnist_model_noise

import numpy as np
import torch
import argparse
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.default import _C as config
from mcrgan.default import update_config
from utils.utils import sort_dataset, compute_accuracy, extract_features
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tqdm import tqdm
import os
from mcrgan.loss import MCRGANloss
import random
random.seed(0)

In [2]:
# data_preprocessing and parameters

device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
DATASET = 'mnist'
ROOT    = 'pth/to/the/dataset'
BATCH_SIZE = 2048
WORKERS = 8
DATA_SAMPLE = 50000
LABEL = 4

dataloader, dataset = get_dataloader(
        data_name=DATASET,
        root=ROOT,
        batch_size=BATCH_SIZE,
        num_workers=WORKERS)


In [3]:
import collections

def get_netG(netD,netV,net_noise,radius=1):
    models = [netD, netV,net_noise]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    fed_state_dict = collections.OrderedDict()
    for key in weight_keys:
        fed_state_dict[key] = worker_state_dict[0][key] + radius*worker_state_dict[2][key]*torch.log(1+torch.exp(worker_state_dict[1][key]))
    return fed_state_dict

# new netG already optimized by adam and time learning rate, so we only using netG to update netV
def update_netV(netV,netG,net_noise,fed_state_dict,radius=1):
    models =[netV,netG,net_noise]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    update_dict = collections.OrderedDict()
    for key in weight_keys:
        update_dict[key] = worker_state_dict[0][key] - (worker_state_dict[1][key]-fed_state_dict[key])*(worker_state_dict[2][key]*radius)/torch.log(1+torch.exp(-worker_state_dict[0][key]))
    return update_dict
        

In [4]:
#models
# train hyperparameters, batch norm not used var
#store_Var = []
import time
from torch_mimicry.training import scheduler, logger, metric_log

from test_acc import nearsub
import numpy as np
import torch
import argparse
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.default import _C as config
from mcrgan.default import update_config
from utils.utils import sort_dataset, compute_accuracy, extract_features2
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tqdm import tqdm
import os

start_time = time.time()
#global_step = 0
num_steps = 150
n_dis = 1
n_dis2 = 1
numclasses =10
lr1 = 1e-3
lr2 = 1e-3
log_data = metric_log.MetricLog()
data_time = time.time()
iter_dataloader = iter(dataloader)
data, label = next(iter_dataloader)
data_time = time.time() - data_time
real_cpu = data.to(device)
real_label = label.clone().detach()

netD, netG, netV = get_BNN_models(DATASET, device, LABEL)
net_noise = get_noise(DATASET, device, LABEL)
fed_state_dict = get_netG(netD,netV,net_noise)
netG.load_state_dict(fed_state_dict)
optD = optim.Adam(netD.parameters(), lr1, betas=(0.5, 0.999))
optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))

Acc_results_D = []
Acc_results_G = []




transform = transforms.Compose(
            [transforms.Resize(32),
             transforms.ToTensor(),
             transforms.Normalize(0.5, 0.5)])

trainset = datasets.MNIST(root=ROOT, train=True, download=True, transform=transform)
testset = datasets.MNIST(root=ROOT, train=False, download=True, transform=transform)

testloader = torch.utils.data.DataLoader(
        testset, batch_size=DATA_SAMPLE, shuffle=False, num_workers=WORKERS)

trainloader = torch.utils.data.DataLoader(
        trainset, batch_size=DATA_SAMPLE, shuffle=False, num_workers=WORKERS)
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
n_comp = 10


In [5]:
netD, netG, netV

(DataParallel(
   (module): DiscriminatorMNIST_nz11(
     (main): Sequential(
       (0): Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
       (1): LeakyReLU(negative_slope=0.2, inplace=True)
       (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
       (3): LeakyReLU(negative_slope=0.2, inplace=True)
       (4): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
       (5): LeakyReLU(negative_slope=0.2, inplace=True)
       (6): Conv2d(256, 11, kernel_size=(4, 4), stride=(1, 1), bias=False)
       (7): Flatten(start_dim=1, end_dim=-1)
     )
   )
 ),
 DataParallel(
   (module): DiscriminatorMNIST_nz11(
     (main): Sequential(
       (0): Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
       (1): LeakyReLU(negative_slope=0.2, inplace=True)
       (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
       (3): LeakyReLU(negative_s

In [6]:
net_noise

DataParallel(
  (module): DiscriminatorMNIST_nz11(
    (main): Sequential(
      (0): Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): LeakyReLU(negative_slope=0.2, inplace=True)
      (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (3): LeakyReLU(negative_slope=0.2, inplace=True)
      (4): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (5): LeakyReLU(negative_slope=0.2, inplace=True)
      (6): Conv2d(256, 11, kernel_size=(4, 4), stride=(1, 1), bias=False)
      (7): Flatten(start_dim=1, end_dim=-1)
    )
  )
)

In [7]:
# golden search algorithm to determined the radius of the sample ball, due to traditional updating netV is too slow
# no meaning for n_dis =2 for our case, just to have output for every update
# sample too much would arise cuda memory error
def gs_radius(radius1,radius2,netD,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=1,gam3=1):
    mcr_gan_loss=MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=11, rho=None)
    #optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
    
    Z = netD(real_cpu)
    net_noise = get_noise(DATASET, device, LABEL)
    err1 = 0
    err2 = 1000
    while abs(err1 - err2)> tor or abs(radius1-radius2)>tor:
        a1 = radius2 - lam*(radius2-radius1)
        a2 = radius1 + lam*(radius2-radius1)
        fed_state_dict = get_netG(netD,netV,net_noise,a1)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err1 = errG1
        
        fed_state_dict = get_netG(netD,netV,net_noise,a2)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err2 = errG2
        
        if err1 > err2:
            radius1 = a1
            a1 = a2
            err1 = err2
            a2 = radius1 + lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a2)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err2 = errG2
            #print('r1',radius1)
            #print(errG2, errG_EC2)
        else:
            radius2 = a2
            a2 = a1
            a1 = radius2 - lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a1)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err1 = errG1
            #print('r2',radius2)
            #print(errG1, errG_EC1)
         
    return radius1,radius2

In [8]:
# fixed radius case
mean_V = []
n_dis2 = 1
n_dis = 1
mode = 11
gam3 = 1
mcr_gan_loss = MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)
loss_V = 0
last_V = -1000
num_steps = 500
lr1 = 1e-3
lr2 = 1e-3
optD = optim.Adam(netD.parameters(), lr1, betas=(0.5, 0.999))
optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
meanV_sum = []
radius_list = []
radius = 1
for epoch in range(num_steps):

    for i in range(n_dis):
        netD.zero_grad()
        optD.zero_grad()
        Z = netD(real_cpu)
        net_noise = get_noise(DATASET, device, LABEL)
        radius,_ = gs_radius(radius1=0,radius2=100,netD=netD,real_cpu=real_cpu,real_label=real_label,tor=1e-5,n_dis=2,lam=0.618,mode=1)
        radius_list.append(radius)
        print(radius)
        fed_state_dict = get_netG(netD,netV,net_noise,radius)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errD, errD_EC = mcr_gan_loss(Z, Z_bar, real_label, 1, n_dis)
        print(errD, errD_EC)
        errD.backward()
        optD.step()
    temp = 0
    for j in range(n_dis2):
        netG.zero_grad()
        optG.zero_grad()

        Z = netD(real_cpu)
        net_noise = get_noise(DATASET, device, LABEL)
        radius,_ = gs_radius(radius1=0,radius2=100,netD=netD,real_cpu=real_cpu,real_label=real_label,tor=1e-5,n_dis=2,lam=0.618,mode=1)
        radius_list.append(radius)
        print(radius)
        fed_state_dict = get_netG(netD,netV,net_noise,radius)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG, errG_EC = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        print(errG, errG_EC)
        temp = errG
        errG = (-1) * errG
        errG.backward()
        optG.step()

        update_dic = update_netV(netV,netG,net_noise,fed_state_dict,radius)
        netV.load_state_dict(update_dic)

        
    meanV_num = []
    
    tempV     = 0
    for key in update_dic.keys():
        meanV_num.append(torch.mean(update_dic[key]))
        tempV += torch.sum(update_dic[key])
    
    mean_V.append(meanV_num)
    meanV_sum.append(tempV)
    #print(meanV_num)
    #print(tempV)
    
    

2.7166754794684667
tensor(-3.0280, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-0.5444, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.5618, device='cuda:0', grad_fn=<NegBackward0>), tensor(-5.8331, device='cuda:0', grad_fn=<AddBackward0>)]
7.055057346773317
tensor(-2.5346, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-0.1398, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.3187, device='cuda:0', grad_fn=<NegBackward0>), tensor(-6.9733, device='cuda:0', grad_fn=<AddBackward0>)]
99.99100627058222
tensor(-2.7304, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-0.1398, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.4653, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.3750, device='cuda:0', grad_fn=<AddBackward0>)]
13.948120984091018
tensor(-2.8185, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-0.1982, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.3532, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.6509, device='cuda:0', grad_fn=<AddBackward0>)

10.23723435757005
tensor(-4.4648, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-2.0615, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.3550, device='cuda:0', grad_fn=<NegBackward0>), tensor(-6.7549, device='cuda:0', grad_fn=<AddBackward0>)]
99.9430696104917
tensor(-4.5131, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-2.0615, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.3687, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.0270, device='cuda:0', grad_fn=<AddBackward0>)]
11.148192343784164
tensor(-4.7823, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-2.2636, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.3994, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.3254, device='cuda:0', grad_fn=<AddBackward0>)]
99.95449078969926
tensor(-4.7271, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-2.2636, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.3466, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.3055, device='cuda:0', grad_fn=<AddBackward0>)]


99.95472118924262
tensor(-5.0040, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-2.5851, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.2494, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.7543, device='cuda:0', grad_fn=<AddBackward0>)]
99.97599053539507
tensor(-5.4584, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-2.9149, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.4070, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.4698, device='cuda:0', grad_fn=<AddBackward0>)]
2.589567755901023
tensor(-6.2165, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-2.9149, device='cuda:0', grad_fn=<NegBackward0>), tensor(-2.0912, device='cuda:0', grad_fn=<NegBackward0>), tensor(-2.3550, device='cuda:0', grad_fn=<AddBackward0>)]
11.912407253043764
tensor(-5.0215, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-2.5897, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.2241, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.0950, device='cuda:0', grad_fn=<AddBackward0>)]

21.316634729216236
tensor(-5.5605, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-2.9327, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.4768, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.5938, device='cuda:0', grad_fn=<AddBackward0>)]
3.0848243395170907
tensor(-6.1714, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-2.9327, device='cuda:0', grad_fn=<NegBackward0>), tensor(-1.6980, device='cuda:0', grad_fn=<NegBackward0>), tensor(-3.6675, device='cuda:0', grad_fn=<AddBackward0>)]
2.801272594436937
tensor(-5.9830, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-2.9893, device='cuda:0', grad_fn=<NegBackward0>), tensor(-1.7909, device='cuda:0', grad_fn=<NegBackward0>), tensor(-2.3294, device='cuda:0', grad_fn=<AddBackward0>)]
99.84926736243102
tensor(-5.6375, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-2.9893, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.4730, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.8042, device='cuda:0', grad_fn=<AddBackward0>)

2.454626798102609
tensor(-6.6001, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-3.2771, device='cuda:0', grad_fn=<NegBackward0>), tensor(-2.3854, device='cuda:0', grad_fn=<NegBackward0>), tensor(-1.5539, device='cuda:0', grad_fn=<AddBackward0>)]
15.765015499083932
tensor(-5.7989, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-3.2621, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.3606, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.8135, device='cuda:0', grad_fn=<AddBackward0>)]
9.39227535115688
tensor(-5.8861, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-3.2621, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.4561, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.7406, device='cuda:0', grad_fn=<AddBackward0>)]
99.91196701014418
tensor(-5.9493, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-3.3794, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.3520, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.1882, device='cuda:0', grad_fn=<AddBackward0>)]


...

55.2993419826817
tensor(-6.7943, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-4.3166, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.2303, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.4631, device='cuda:0', grad_fn=<AddBackward0>)]
99.9678804406981
tensor(-6.9054, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-4.3158, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.3449, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.4376, device='cuda:0', grad_fn=<AddBackward0>)]
0.0312517211281427
tensor(-8.6315, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-4.3158, device='cuda:0', grad_fn=<NegBackward0>), tensor(-4.3156, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.7248e-05, device='cuda:0', grad_fn=<AddBackward0>)]
0.0038128167002186215
tensor(-8.6323, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-4.3162, device='cuda:0', grad_fn=<NegBackward0>), tensor(-4.3162, device='cuda:0', grad_fn=<NegBackward0>), tensor(-4.7684e-06, device='cuda:0', grad_fn=<AddBa

99.97019338491165
tensor(-6.8155, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-4.3164, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.2413, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.5627, device='cuda:0', grad_fn=<AddBackward0>)]
0.0023816647638572903
tensor(-8.6327, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-4.3164, device='cuda:0', grad_fn=<NegBackward0>), tensor(-4.3164, device='cuda:0', grad_fn=<NegBackward0>), tensor(-9.5367e-07, device='cuda:0', grad_fn=<AddBackward0>)]
44.98153847990874
tensor(-6.9695, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-4.3170, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.3875, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.6320, device='cuda:0', grad_fn=<AddBackward0>)]
0.0011858602931858538
tensor(-8.6340, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-4.3170, device='cuda:0', grad_fn=<NegBackward0>), tensor(-4.3170, device='cuda:0', grad_fn=<NegBackward0>), tensor(0., device='cuda:0', grad_fn=<AddBackwa

0.009200625679647901
tensor(-8.6358, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-4.3179, device='cuda:0', grad_fn=<NegBackward0>), tensor(-4.3179, device='cuda:0', grad_fn=<NegBackward0>), tensor(-5.7220e-06, device='cuda:0', grad_fn=<AddBackward0>)]
0.00110393531345547
tensor(-8.6358, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-4.3179, device='cuda:0', grad_fn=<NegBackward0>), tensor(-4.3179, device='cuda:0', grad_fn=<NegBackward0>), tensor(0., device='cuda:0', grad_fn=<AddBackward0>)]
0.004040230093354864
tensor(-8.6358, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-4.3179, device='cuda:0', grad_fn=<NegBackward0>), tensor(-4.3179, device='cuda:0', grad_fn=<NegBackward0>), tensor(-2.8610e-06, device='cuda:0', grad_fn=<AddBackward0>)]
0.0004216153116509791
tensor(-8.6359, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-4.3180, device='cuda:0', grad_fn=<NegBackward0>), tensor(-4.3180, device='cuda:0', grad_fn=<NegBackward0>), tensor(9.5367e-07, device='cuda:0', grad_fn

0.0050447267356781185
tensor(-8.6363, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-4.3182, device='cuda:0', grad_fn=<NegBackward0>), tensor(-4.3182, device='cuda:0', grad_fn=<NegBackward0>), tensor(-9.5367e-07, device='cuda:0', grad_fn=<AddBackward0>)]
0.004342291875195303
tensor(-8.6363, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-4.3182, device='cuda:0', grad_fn=<NegBackward0>), tensor(-4.3182, device='cuda:0', grad_fn=<NegBackward0>), tensor(-2.8610e-06, device='cuda:0', grad_fn=<AddBackward0>)]
40.570080609457875
tensor(-6.9336, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-4.3183, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.3584, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.5533, device='cuda:0', grad_fn=<AddBackward0>)]
0.0012950718579218853
tensor(-8.6366, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-4.3183, device='cuda:0', grad_fn=<NegBackward0>), tensor(-4.3183, device='cuda:0', grad_fn=<NegBackward0>), tensor(2.8610e-06, device='cuda:0', gr

0.10714184175957941
tensor(-8.4683, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-4.2343, device='cuda:0', grad_fn=<NegBackward0>), tensor(-4.2333, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.0007, device='cuda:0', grad_fn=<AddBackward0>)]
99.9242106140188
tensor(-6.7095, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-4.2673, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.1881, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.5264, device='cuda:0', grad_fn=<AddBackward0>)]
99.97201769495192
tensor(-6.7734, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-4.2673, device='cuda:0', grad_fn=<NegBackward0>), tensor(-0.2684, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.3722, device='cuda:0', grad_fn=<AddBackward0>)]
0.01602615445426053
tensor(-8.5676, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-4.2838, device='cuda:0', grad_fn=<NegBackward0>), tensor(-4.2838, device='cuda:0', grad_fn=<NegBackward0>), tensor(-1.2398e-05, device='cuda:0', grad_fn=<AddBackwa

In [9]:
radius_list

[2.7166754794684667,
 7.055057346773317,
 99.99100627058222,
 13.948120984091018,
 25.81796724923468,
 99.97646287432343,
 5.837915463543,
 99.98522365678853,
 99.9892983703627,
 16.821590696964066,
 25.759401470647134,
 9.959821075067943,
 99.9943565598953,
 3.6586742737109623,
 99.99485277989193,
 99.96959799936188,
 9.757885734006214,
 26.014535296977737,
 3.3482202649558954,
 99.99429320794465,
 99.98462908347251,
 3.3475126073441475,
 99.97150719603785,
 2.9025517681223825,
 99.95508617524904,
 2.3496149886078967,
 18.963910019281215,
 17.97815156461361,
 13.569805929267124,
 2.69096549320803,
 99.98811531053728,
 2.9639493770765695,
 2.677335193036056,
 10.23723435757005,
 99.9430696104917,
 11.148192343784164,
 99.95449078969926,
 99.97513579557263,
 99.97734117787088,
 99.99338824107328,
 98.74756945849492,
 3.078302200763846,
 99.9892983703627,
 99.91187451324514,
 26.427534030497217,
 2.5971473529951337,
 10.839819540264026,
 99.99526487246054,
 2.3706328551240614,
 99.985198

In [10]:
#acc at first for NetD
train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(testloader, netD, netG)

test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netD, netG)


print("----------------------------")
print("Train_z, Test_z")
nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

#acc at first for NetD
from sklearn.neighbors import KNeighborsClassifier 
knn= KNeighborsClassifier(n_neighbors=10) 

knn.fit(train_Z,train_labels) 
knn.score(test_Z,test_labels) 
 
y_predict = knn.predict(test_Z) 

count =0
for k in range(10000):
    if y_predict[k] == test_labels[k]:
        count +=1
print('NetD, Acc at first',count/10000)
Acc_results_D.append(count/10000)

#acc at first for NetG
train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

print("----------------------------")
print("Train_z, Test_z")
nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)


#acc at first
from sklearn.neighbors import KNeighborsClassifier 
knn= KNeighborsClassifier(n_neighbors=10) 

knn.fit(train_Z,train_labels) 
knn.score(test_Z,test_labels) 
 
y_predict = knn.predict(test_Z) 

count =0
for k in range(10000):
    if y_predict[k] == test_labels[k]:
        count +=1
print('NetG, Acc at first',count/10000)
Acc_results_G.append(count/10000)


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.70s/it]


----------------------------
Train_z, Test_z
PCA: 0.4475
SVD: 0.2923
NetD, Acc at first 0.9728


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.67s/it]


----------------------------
Train_z, Test_z
PCA: 0.5636
SVD: 0.29300000000000004
NetG, Acc at first 0.9711


In [11]:
#sample the length of radius
L_list= []
for _ in range(20):
    r1,r2 = gs_radius(0,100,netD,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_list.append(r1)
print(max(L_list),min(L_list),np.mean(L_list),np.var(L_list))

0.03109852491045105
0
99.9755074135736
0
1.6728968139816078
0.0005953855497773813
0.0015589110654930858
99.87040215989782
0.0005953855497773813
0.01476858859542767
0.0005953855497773813
11.565786124653174
39.49075954913079
0
17.98837562677624
99.98211347073463
0.0021542966152704674
99.98775523533722
0.0005953855497773813
0.0066047428669414095
99.98775523533722 0 23.52960815001689 1545.4996652310451
